## Data recovery

In [11]:
from sqlalchemy.orm import sessionmaker
from models import City, DbInteraction
from datetime import date, datetime
from sqlalchemy import select
from csv import DictReader

In [12]:
db = DbInteraction('localhost,5432', 'mex_covid_py','postgres', 'this_isAstrong#password')
Session = sessionmaker(bind=db.engine)
ses = Session()

In [13]:
cities = {}
daily_city_summaries = []

In [14]:
result = ses.execute(select(City))
for city in result.scalars().all():
    cities[(city.city_code, city.state_id)] = {'confirmed': 0, 'deaths': 0, 'c_name': city.city_name, 'lat': city.latitude, 'lon': city.longitude}

In [15]:
def get_csv_date(file_name):
    return datetime.strptime(file_name[:6], '%y%m%d').date()

In [16]:
def get_indicators(cities, folder, file):
    file_date = get_csv_date(file)
    change_format = date(2020,10,7)
    with open(f'{folder}/{file}', 'r', encoding="ISO-8859-1") as csv_f:
            f = DictReader(csv_f)
            for row in f:
                city_index = (int(row['MUNICIPIO_RES']),
                              int(row['ENTIDAD_RES']))
                if city_index in cities:
                    if file_date < change_format:
                        if int(row['RESULTADO']) == 1:
                            cities[city_index]['confirmed'] += 1
                            if row['FECHA_DEF'] != '9999-99-99':
                                cities[city_index]['deaths'] += 1
                    else:
                        if int(row['CLASIFICACION_FINAL']) in (1,2,3):
                            cities[city_index]['confirmed'] += 1
                            if row['FECHA_DEF'] != '9999-99-99':
                                cities[city_index]['deaths'] += 1
        return cities

## Data visualization

In [17]:
import folium

In [18]:
cities_indicators = get_indicators(cities, 'zip', '210721COVID19MEXICO.csv')

In [19]:
m = folium.Map(location=[23.634501, -102.552784], tiles="OpenStreetMap", zoom_start=5)

In [21]:
# Create the points
for city in cities_indicators.values():
    if city['confirmed'] > 100:
        folium.Circle(
            location=[city['lat'], city['lon']],
            popup=f'{city["c_name"].title()} confirmed:{city["confirmed"]}',
            radius=float(city['confirmed']),
            color='crimson',
            fill=True,
            fill_color='crimson'
        ).add_to(m)
# Show map
m